In [1]:
import os
from pathlib import Path

In [2]:
import tensorflow as tf

In [3]:
vg_16 = tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [4]:
# Update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen=True)
class PrepareBaseModelConfig:
  root_dir: Path
  base_model_path: Path
  updated_base_model_path: Path
  params_image_size: list
  params_learning_rate: float
  params_include_top: bool
  params_weights: str
  params_classes: int

In [5]:
# Update configuration manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_path = PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
    
    def prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config =self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        base_model_config = PrepareBaseModelConfig(
            root_dir=config.root_dir,
            base_model_path=config.base_model_path,
            updated_base_model_path=config.updated_model_path,
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            
        )
        
        return base_model_config

In [7]:
# Create components

import os
import zipfile
import tensorflow as tf

In [19]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def download(self):
        '''
        Fetch data from G-Drive
        '''
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            input_shape=self.config.params_image_size
        )
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    def update(self):
        self.updated_model=self._update_model(freeze_all=True,
                                            freeze_till=0,
                                            model=self.model,
                                            classes=self.config.params_classes,
                                            learning_rate=self.config.params_learning_rate)
        self.save_model(path=self.config.updated_base_model_path, model=self.updated_model)
    
    @staticmethod
    def _update_model(freeze_all: bool, 
                      freeze_till:int,
                      model:tf.keras.Model,
                      classes:int,
                      learning_rate:float):
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:freeze_till]:
                layer.trainable = True
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        updated_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        updated_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        updated_model.summary()
        return updated_model
        
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

In [20]:
# Updating pipeline
from cnnClassifier.utils import logger
try:
    config = ConfigurationManager()
    base_model_config = config.prepare_base_model_config()
    base_model = PrepareBaseModel(base_model_config)
    base_model.download()
    base_model.update()
    logger.info(f"Base model pipeline succesful!")
except Exception as e:
    raise e

[2023-12-19 15:04:14,642: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-19 15:04:14,643: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-19 15:04:14,854: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2023-12-19 15:04:15,383: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: artifacts/base_model/raw/vgg16.hd5\assets
[2023-12-19 15:04:15,645: INFO: builder_impl: Assets written to: artifacts/base_model/raw/vgg16.hd5\assets]
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Sha

In [11]:
cd ../

d:\data_science\kidney_disease_classification


d:\data_science\kidney_disease_classification\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
